# Lab 05-a: POS Tagging with NLTK and spaCy

- **NLTK**: Taggers: PerceptronTagger, Brill Tagger, etc.
- **spaCy**: Industrial-strength NLP framework

## 1. Setup và Installation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import time

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('brown')
nltk.download('universal_tagset')
nltk.download('maxent_ne_chunker')
nltk.download('words')

from nltk import pos_tag, word_tokenize
from nltk.corpus import brown
from nltk.tag import PerceptronTagger, UnigramTagger, BigramTagger, TrigramTagger



In [ ]:
!pip install spacy

In [ ]:
import spacy

In [ ]:
!python -m spacy download en_core_web_sm # for small model
# Run: python -m spacy download en_core_web_md # for medium model

## 2. NLTK POS Taggers

### 2.1. Default POS Tagger (Perceptron-based)

In [ ]:
# NLTK's default tagger (PerceptronTagger)
test_sentences = [
    "The quick brown fox jumps over the lazy dog",
    "I want to learn natural language processing",
    "Python is a popular programming language",
    "She sells seashells by the seashore",
    "The cat sat on the mat"
]

print("NLTK Default POS Tagger (Penn Treebank Tags)")

for sentence in test_sentences:
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)
    
    print(f"\n'{sentence}'")
    print("  Tags:", end=" ")
    for word, tag in pos_tags:
        print(f"{word}/{tag}", end=" ")
    print()

In [ ]:
# Using Universal tagset (simplified)
print("NLTK POS Tagger (Universal Tagset)")

for sentence in test_sentences:
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens, tagset='universal')
    
    print(f"\n'{sentence}'")
    print("  Tags:", end=" ")
    for word, tag in pos_tags:
        print(f"{word}/{tag}", end=" ")
    print()

### 2.2. Penn Treebank vs Universal Tagset

**Penn Treebank**: 36 tags (very detail)
- NN, NNS, NNP, NNPS (noun variations)
- VB, VBD, VBG, VBN, VBP, VBZ (verb variations)

**Universal**: 12 tags (much more simplified)
- NOUN, VERB, ADJ, ADV, etc.

In [ ]:
# Tag comparison
sample_text = "The cats are running quickly through the beautiful garden"
tokens = word_tokenize(sample_text)

penn_tags = pos_tag(tokens)
universal_tags = pos_tag(tokens, tagset='universal')

print("\nTagset Comparison:")
print(f"{'Word':<15} {'Penn Treebank':<15} {'Universal':<15}")
print("-" * 45)

for (w1, pt), (w2, ut) in zip(penn_tags, universal_tags):
    print(f"{w1:<15} {pt:<15} {ut:<15}")

### 2.3. Training Custom NLTK Taggers

NLTK allows to train custom taggers

In [ ]:
# Load Brown corpus for training
print("Loading Brown corpus...")
brown_tagged = brown.tagged_sents(tagset='universal')

# Split data
split_idx = int(len(brown_tagged) * 0.8)
train_sents = brown_tagged[:split_idx]
test_sents = brown_tagged[split_idx:split_idx + 1000]  # Use 1000 for testing

print(f"Training sentences: {len(train_sents):,}")
print(f"Test sentences: {len(test_sents):,}")

In [ ]:
# 1. Unigram Tagger
print("\n[1/3] Training Unigram Tagger...")
unigram_tagger = UnigramTagger(train_sents)
unigram_accuracy = unigram_tagger.accuracy(test_sents)
print(f"  Accuracy: {unigram_accuracy:.2%}")

In [ ]:
# 2. Bigram Tagger with Unigram backoff
print("\n[2/3] Training Bigram Tagger (with Unigram backoff)...")
bigram_tagger = BigramTagger(train_sents, backoff=unigram_tagger)
bigram_accuracy = bigram_tagger.accuracy(test_sents)
print(f"  Accuracy: {bigram_accuracy:.2%}")

In [ ]:
# 3. Trigram Tagger with backoff chain
print("\n[3/3] Training Trigram Tagger (with Bigram & Unigram backoff)...")
trigram_tagger = TrigramTagger(train_sents, backoff=bigram_tagger)
trigram_accuracy = trigram_tagger.accuracy(test_sents)
print(f"  Accuracy: {trigram_accuracy:.2%}")

In [ ]:
# Compare taggers on sample sentence
sample = "The quick brown fox jumps over the lazy dog".split()

print("Comparison on sample sentence:")
print(f"Sentence: {' '.join(sample)}")

print("Unigram:", unigram_tagger.tag(sample))
print("Bigram: ", bigram_tagger.tag(sample))
print("Trigram:", trigram_tagger.tag(sample))

## 3. spaCy POS Tagging

spaCy is an industrial-strength NLP framework:
- Pre-trained models
- NER, dependency parsing, etc.

### 3.1. Load spaCy Models

In [ ]:
# Load spaCy model
# Small model (11MB)
print("Loading spaCy model (en_core_web_sm)...")
try:
    nlp_sm = spacy.load("en_core_web_sm")
    print("  Small model loaded")
except:
    print("  Small model not found. Run: python -m spacy download en_core_web_sm")
    nlp_sm = None

# Medium model (40MB) - better accuracy
try:
    nlp_md = spacy.load("en_core_web_md")
    print("  Medium model loaded")
except:
    print("  Medium model not found. Run: python -m spacy download en_core_web_md")
    nlp_md = None

### 3.2. Basic POS Tagging with spaCy

In [ ]:
if nlp_sm:
    print("spaCy POS Tagging")
    
    for sentence in test_sentences:
        doc = nlp_sm(sentence)
        
        print(f"\n'{sentence}'")
        print(f"  {'Word':<15} {'POS':<10} {'Tag':<10} {'Dep':<10}")
        
        for token in doc:
            print(f"  {token.text:<15} {token.pos_:<10} {token.tag_:<10} {token.dep_:<10}")

### 3.3. spaCy Tag Explanations

In [ ]:
if nlp_sm:
    # Explain tags
    sample_text = "The cats were running quickly through the beautiful garden"
    doc = nlp_sm(sample_text)
    
    print("\nDetailed Tag Information:")
    print(f"{'Word':<15} {'POS':<10} {'Tag':<10} {'Explanation':<30}")
    
    for token in doc:
        # Get explanation
        explanation = spacy.explain(token.tag_)
        print(f"{token.text:<15} {token.pos_:<10} {token.tag_:<10} {explanation if explanation else 'N/A':<30}")

### 3.4. Advanced spaCy Features

In [ ]:
if nlp_sm:
    # Morphological features
    text = "He was running quickly, but she runs faster."
    doc = nlp_sm(text)
    
    print("\nMorphological Analysis:")
    print(f"{'Word':<15} {'Lemma':<15} {'POS':<10} {'Morph Features':<30}")
    
    for token in doc:
        print(f"{token.text:<15} {token.lemma_:<15} {token.pos_:<10} {str(token.morph):<30}")

## 4. Comparison and Benchmarking

- Accuracy
- Speed
- Ease of use

### 4.1. Speed Comparison

In [ ]:
# Prepare test data
test_texts = [
    "The quick brown fox jumps over the lazy dog.",
    "Natural language processing is fascinating.",
    "Machine learning models can understand human language.",
    "Python is a versatile programming language.",
    "Deep learning has revolutionized artificial intelligence."
] * 100  # 500 sentences

print(f"Testing on {len(test_texts)} sentences...\n")

results = {}

# NLTK Default
print("[1/3] Testing NLTK default tagger...")
start = time.time()
for text in test_texts:
    tokens = word_tokenize(text)
    _ = pos_tag(tokens)
nltk_time = time.time() - start
results['NLTK Default'] = nltk_time
print(f"  Time: {nltk_time:.3f}s ({len(test_texts)/nltk_time:.1f} sent/s)")

# spaCy Small
if nlp_sm:
    print("\n[2/3] Testing spaCy (small model)...")
    start = time.time()
    for text in test_texts:
        _ = nlp_sm(text)
    spacy_sm_time = time.time() - start
    results['spaCy Small'] = spacy_sm_time
    print(f"  Time: {spacy_sm_time:.3f}s ({len(test_texts)/spacy_sm_time:.1f} sent/s)")

# spaCy Medium
if nlp_md:
    print("\n[3/3] Testing spaCy (medium model)...")
    start = time.time()
    for text in test_texts:
        _ = nlp_md(text)
    spacy_md_time = time.time() - start
    results['spaCy Medium'] = spacy_md_time
    print(f"  Time: {spacy_md_time:.3f}s ({len(test_texts)/spacy_md_time:.1f} sent/s)")

In [ ]:
# Visualize speed comparison
if results:
    plt.figure(figsize=(10, 6))
    models = list(results.keys())
    times = list(results.values())
    
    plt.bar(models, times, color=['#3498db', '#2ecc71', '#f39c12'][:len(models)])
    plt.ylabel('Time (seconds)', fontsize=12)
    plt.title(f'Speed Comparison ({len(test_texts)} sentences)', fontsize=14, fontweight='bold')
    plt.xticks(rotation=15)
    
    # Add value labels
    for i, (model, time_val) in enumerate(zip(models, times)):
        plt.text(i, time_val + 0.1, f'{time_val:.2f}s', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()

### 4.2. Side-by-Side Comparison

In [ ]:
# Compare different taggers on the same sentences
comparison_sentences = [
    "The stock market crashed yesterday.",
    "She can run faster than anyone.",
    "I will book a flight to Paris.",
    "The book on the table is mine."
]

for sent in comparison_sentences:
    print(f"Sentence: {sent}")
    
    tokens = word_tokenize(sent)
    
    # NLTK
    nltk_tags = pos_tag(tokens, tagset='universal')
    print(f"\nNLTK:    ", end="")
    for word, tag in nltk_tags:
        print(f"{word}/{tag}", end=" ")
    
    # Unigram
    unigram_tags = unigram_tagger.tag(tokens)
    print(f"\nUnigram: ", end="")
    for word, tag in unigram_tags:
        tag = tag if tag else 'NOUN'
        print(f"{word}/{tag}", end=" ")
    
    # Trigram
    trigram_tags = trigram_tagger.tag(tokens)
    print(f"\nTrigram: ", end="")
    for word, tag in trigram_tags:
        tag = tag if tag else 'NOUN'
        print(f"{word}/{tag}", end=" ")
    
    # spaCy
    if nlp_sm:
        doc = nlp_sm(sent)
        print(f"\nspaCy:   ", end="")
        for token in doc:
            print(f"{token.text}/{token.pos_}", end=" ")

## 5. Examples

### 5.1. Extract Noun Phrases

In [ ]:
# Extract noun phrases using POS patterns
def extract_noun_phrases_nltk(text):
    """
    Extract noun phrases (ADJ* NOUN+) using NLTK.
    """
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens, tagset='universal')
    
    noun_phrases = []
    current_phrase = []
    
    for word, tag in pos_tags:
        if tag in ['ADJ', 'NOUN']:
            current_phrase.append(word)
        else:
            if current_phrase and any(w for w, t in pos_tags if t == 'NOUN' and w in current_phrase):
                noun_phrases.append(' '.join(current_phrase))
            current_phrase = []
    
    if current_phrase:
        noun_phrases.append(' '.join(current_phrase))
    
    return noun_phrases

sample_text = """
The quick brown fox jumps over the lazy dog. 
Natural language processing is a fascinating field of artificial intelligence.
Modern machine learning algorithms can process large amounts of textual data.
"""

print("Extracting noun phrases with NLTK:")
nltk_phrases = extract_noun_phrases_nltk(sample_text)
for phrase in nltk_phrases:
    if len(phrase.split()) > 1:  # Only multi-word phrases
        print(f"  - {phrase}")

In [ ]:
# Extract noun phrases using spaCy
if nlp_sm:
    print("\nExtracting noun phrases with spaCy:")
    doc = nlp_sm(sample_text)
    
    for chunk in doc.noun_chunks:
        print(f"  - {chunk.text} (root: {chunk.root.text}, dep: {chunk.root.dep_})")

### 5.2. Extract Verbs and Their Objects

In [ ]:
if nlp_sm:
    # Extract verb-object pairs using dependency parsing
    text = """I love programming. She studies natural language processing. 
    They built a sophisticated machine learning model."""
    
    doc = nlp_sm(text)
    
    print("Verb-Object pairs:")
    for token in doc:
        if token.pos_ == "VERB":
            # Find direct objects
            objects = [child for child in token.children if child.dep_ in ('dobj', 'obj')]
            for obj in objects:
                # Get the full noun phrase
                obj_phrase = " ".join([child.text for child in obj.subtree])
                print(f"  {token.text} → {obj_phrase}")

### 5.3. Text Simplification

In [ ]:
# Remove adjectives and adverbs for text simplification
def simplify_text(text, use_spacy=True):
    """
    Remove adjectives and adverbs to simplify text.
    """
    if use_spacy and nlp_sm:
        doc = nlp_sm(text)
        simplified = " ".join([token.text for token in doc 
                              if token.pos_ not in ['ADJ', 'ADV']])
    else:
        tokens = word_tokenize(text)
        pos_tags = pos_tag(tokens, tagset='universal')
        simplified = " ".join([word for word, tag in pos_tags 
                              if tag not in ['ADJ', 'ADV']])
    
    return simplified

complex_text = "The extremely talented young programmer quickly developed a highly sophisticated algorithm."

print("Original text:")
print(f"  {complex_text}")
print("\nSimplified (NLTK):")
print(f"  {simplify_text(complex_text, use_spacy=False)}")

if nlp_sm:
    print("\nSimplified (spaCy):")
    print(f"  {simplify_text(complex_text, use_spacy=True)}")

## 6. Summary

### When to Use:

**Use NLTK when:**
- Learning NLP concepts
- Need fine-grained control
- Want to implement custom algorithms
- Working with smaller datasets

**Use spaCy when:**
- Building production systems
- Need high performance
- Want additional features (NER, dependency parsing)
- Working with large-scale data